# Store parameters value for each station in an xlsx file


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from googletrans import Translator
from tqdm.auto import tqdm

translator = Translator()

In [ ]:
# Get the current directory of the notebook

import os

current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir, os.pardir))
path_files = os.path.join(parent_dir, "phychem_data", "phychem_param_80_19", "all_param")
file_names = os.listdir(path_files)
print(file_names)

In [ ]:
# remove all!!!!
for i, file in enumerate(file_names):
    csv = file.split(".")[1]
    if csv != "csv":
        file_names.pop(i)
file_names

In [ ]:
# fill with othere files
french_name_param = []
code_param = []
list_df_files = []

for i, file_name in enumerate(file_names):

    df = pd.read_csv(os.path.join("phychem_param_80_19","all_param",file_name),  delimiter=";") # correctly chose del
    df["date_prelevement"] = pd.to_datetime(df["date_prelevement"])
    df = df.sort_values("date_prelevement")

    french_name_param += df["libelle_parametre"].unique().tolist()
    code_param += df["code_parametre"].unique().tolist()

    list_df_files.append(df)
    print("df exported")

In [ ]:
toEnglish = {french_name: translator.translate(french_name, src='fr', dest='en').text for french_name in french_name_param}
toEnglish['Orthophosphates (PO4)']= 'Orthophosphates (PO4)'
toEnglish

In [ ]:
name_param = list(toEnglish.values())
name_param

In [ ]:
df_files = pd.concat(list_df_files).reset_index(drop=True)

In [ ]:
date_limite_start = pd.Timestamp('1980-03-01')
date_limite_end = pd.Timestamp('2019-11-30')

df_files = df_files.loc[df_files['date_prelevement'] >= date_limite_start] # only consider from march bc start of hydrological year in france
df_files = df_files.loc[df_files['date_prelevement'] <= date_limite_end].reset_index(drop=True)
df_files.shape


In [ ]:
df_files_grouped = df_files.groupby("code_station").agg({
    "date_prelevement" : list,
    "code_parametre" : list,
    "libelle_parametre" : list,
    "resultat" : list
}).reset_index()
df_files_grouped.head()

In [ ]:
def df_architecture(name_param=name_param):

    column_names = ["Year", "Season"]
    column_names += name_param
    df = pd.DataFrame(columns=column_names)

    list_years = [[i, i, i, i] for i in range(1981,2020)]
    years = [year for sublist in list_years for year in sublist]
    for i in range(3):
        years.insert(0, 1980)
    df["Year"] = years

    list_seasons = [["SP", "SM", "FL", "WT"] for i in range(40)]
    seasons = [year for sublist in list_seasons for year in sublist]
    del seasons[-1]
    df["Season"] = seasons

    null_values = [[] for i in range(len(seasons))]
    for col in df.columns[2::]:
        df[col] = null_values

    return df

    

In [ ]:
season_dict = {
    1 : "WT",
    2 : "WT",
    3 : "SP",
    4 : "SP",
    5 : "SP",
    6 : "SM",
    7 : "SM",
    8 : "SM",
    9 : "FL",
    10 : "FL",
    11 : "FL",
    12 : "WT"
}

In [ ]:
def fill_param(station_id, df_archi, df_files):
    
    serie_station = df_files.loc[df_files["code_station"]==station_id]
    for date, parameter, value in zip(serie_station["date_prelevement"].values[0], serie_station["libelle_parametre"].values[0], serie_station.resultat.values[0]):
        year = date.year
        month =  date.month
        season = season_dict[month]
        # print(year, month, season)

        if month == 12 : year = year+1 # considering december for next year
        else: year=year

        index = df_archi.loc[(df_archi["Year"]==year) & (df_archi["Season"]==season)].index[0]
        df_archi.at[index, toEnglish[parameter]] = df_archi.at[index, toEnglish[parameter]] + [value]
        

    for col in df_archi.columns:
        df_archi[col] = df_archi[col].apply(lambda x: [-1] if isinstance(x, list) and len(x) == 0 else x)

    for col in df_archi.columns:
        df_archi[col] = df_archi[col].apply(lambda x: np.median(np.array(x)) if isinstance(x, list) else x)

    return df_archi


In [ ]:
def export_xlsx(station_id, df):
    file_name = f"station_{station_id}.xlsx"
    file_path = os.path.join("phychem_param_80_19", "param_station_xlsx", file_name)
    df.to_excel(file_path, sheet_name="seasonally", index=False)

In [ ]:
len(df_files_grouped["code_station"])

In [ ]:
for station_id in tqdm(df_files_grouped["code_station"][1646:]):
    
    df_archi = df_architecture(name_param=name_param)
    df = fill_param(station_id, df_archi=df_archi, df_files=df_files_grouped)
    export_xlsx(station_id=station_id, df=df)